<a href="https://colab.research.google.com/github/BVika/Methods_of_semantic_information_processing/blob/main/%D0%9B%D0%B0%D0%B1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная 3



##1 задание. Простейший случай с 1 нейроном

### Создать нейронную сеть с нуля, т.е. не используя готовые библиотеки. Пример работы на любом табличном датасете.


Импортируем библиотреки

In [ ]:
import numpy as np
import pandas as pd

Создаем датасет

1.   Возраст человека
2.   Зарплата человека: 0 - менее 50,000, 1 - более 50,000

Предсказываем с какой вероятностью у человека будет зарплата больее 50000 относительно возраста

In [ ]:
data = {
    'age': [22, 25, 30, 35, 40, 45, 50, 55, 60],
    'income_above_50000': [0, 0, 0, 1, 1, 1, 1, 1, 1]
}
df = pd.DataFrame(data)

Линейная регрессия

In [ ]:

def linear_regression(df: pd.DataFrame) -> list[float]:
    x = df['age'].values
    y = df['income_above_50000'].values

    X = np.c_[np.ones(x.shape[0]), x]  # добавляем столбец единиц для bias
    weights = np.linalg.inv(X.T @ X) @ (X.T @ y)  # вычисляем веса: (X^T * X)^(-1) * X^T * y

    predictions = X @ weights
    return predictions.tolist()

print(linear_regression(df))

[0.10771152044957832, 0.19973462379019657, 0.3531064626912269, 0.5064783015922574, 0.6598501404932877, 0.8132219793943183, 0.9665938182953486, 1.119965657196379, 1.2733374960974093]


Функция активации - сигмоида: 1 / (1 + exp(-x))

In [ ]:
def activation_func(x: list[float]) -> list[float]:

    return (1 / (1 + np.exp(-np.array(x)))).tolist()

Создание нейрона

In [ ]:
def neuron(df: pd.DataFrame):
    temp_result = linear_regression(df)
    result = activation_func(temp_result)
    return result

Запускаем нейрон и выводим результат

In [ ]:
output = neuron(df)

In [ ]:
for age, probability in zip(df['age'], output):
    print(f"Возраст: {age}, Вероятность заработка более 50,000: {probability:.2f}")

Возраст: 22, Вероятность заработка более 50,000: 0.53
Возраст: 25, Вероятность заработка более 50,000: 0.55
Возраст: 30, Вероятность заработка более 50,000: 0.59
Возраст: 35, Вероятность заработка более 50,000: 0.62
Возраст: 40, Вероятность заработка более 50,000: 0.66
Возраст: 45, Вероятность заработка более 50,000: 0.69
Возраст: 50, Вероятность заработка более 50,000: 0.72
Возраст: 55, Вероятность заработка более 50,000: 0.75
Возраст: 60, Вероятность заработка более 50,000: 0.78


# 2 задание

### Сделать класс, в котором реализована возможность задать количество нейронов в скрытом слое и провести обучение.

Импорт библиотек

In [ ]:
import numpy as np

In [ ]:
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):

        self.learning_rate = learning_rate

        self.weights_input_hidden = np.random.rand(input_size, hidden_size) - 0.5
        self.weights_hidden_output = np.random.rand(hidden_size, output_size) - 0.5

        #создаем смещения для скрытого и выходного слоев
        self.bias_hidden = np.random.rand(hidden_size) - 0.5
        self.bias_output = np.random.rand(output_size) - 0.5

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        self.hidden_output = self.sigmoid(np.dot(x, self.weights_input_hidden) + self.bias_hidden)
        return self.sigmoid(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output)

    def backward(self, x, y, output):
        output_error = y - output
        hidden_error = (output_error * self.sigmoid_derivative(output)).dot(self.weights_hidden_output.T)

        # Обновление весов и смещений
        self.weights_hidden_output += self.hidden_output.T.dot(output_error * self.sigmoid_derivative(output)) * self.learning_rate
        self.bias_output += np.sum(output_error * self.sigmoid_derivative(output), axis=0) * self.learning_rate
        self.weights_input_hidden += x.T.dot(hidden_error * self.sigmoid_derivative(self.hidden_output)) * self.learning_rate
        self.bias_hidden += np.sum(hidden_error * self.sigmoid_derivative(self.hidden_output), axis=0) * self.learning_rate

    def train(self, x, y, epochs):
        for epoch in range(epochs):
            output = self.forward(x)
            self.backward(x, y, output)
            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - output))
                print(f'Epoch {epoch}, Loss: {loss:.4f}')

    def predict(self, x):
        return self.forward(x)


### **Создаем** **датасет**:

1. Возраст
2. Опыт
3. Зарплата человека: 0 - менее 50,000, 1 - более 50,000


In [ ]:
data = np.array([
    [25, 1, 0],
    [30, 3, 1],
    [35, 5, 1],
    [40, 10, 1],
    [22, 0, 0],
    [28, 2, 0],
    [32, 4, 1],
    [45, 15, 1],
    [50, 20, 1],
])

Разделение на входные и выходные данные

In [ ]:
X = data[:, :2]  # Возраст и опыт
y = data[:, 2].reshape(-1, 1)  # Заработная плата

## Создание и обучение сети

In [ ]:
nn = SimpleNeuralNetwork(input_size=2, hidden_size=3, output_size=1, learning_rate=0.1)
nn.train(X, y, epochs=10000)

Epoch 0, Loss: 0.2395
Epoch 1000, Loss: 0.0080
Epoch 2000, Loss: 0.0023
Epoch 3000, Loss: 0.0013
Epoch 4000, Loss: 0.0009
Epoch 5000, Loss: 0.0007
Epoch 6000, Loss: 0.0006
Epoch 7000, Loss: 0.0005
Epoch 8000, Loss: 0.0004
Epoch 9000, Loss: 0.0004


Результат

In [ ]:
test_data = np.array([[29, 2], [45, 10]])
predictions = nn.predict(test_data)
print("Predictions:")
print(predictions)

Predictions:
[[0.02465025]
 [0.98809631]]
